In [1]:
import ipywidgets as widgets
import qgrid
from collections import namedtuple
from datetime import date, datetime, timedelta
import os
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [2]:
# get mongodb params from environment
connection_params = {
    'user': 'mcrm_user',
    'password': '!098POIlkj!',
    'host': 'sandbox-0.wgv2x.mongodb.net',
    'port': 57475,
    'namespace': 'twiggins',
}

mlab_uri = 'mongodb+srv://{user}:{password}@{host}/{namespace}?retryWrites=true&w=majority'.format(**connection_params)
mlab_collection = 'clients'

#codec_options=CodecOptions(document_class=RawBSONDocument)
client = MongoClient(mlab_uri,retryWrites=False)
db = client.get_default_database()
collection = db.get_collection(
    mlab_collection
)

In [3]:
def meetingDue(row):
    maps  = {'A+': 90, 'A': 120, 'B': 180, 'C': 360}
    if 'lastMeeting' in row and type(row['lastMeeting']) is str:
        deltas = timedelta(maps[row['segment']])
        dates = datetime.strptime(row['lastMeeting'] or '01/01/00','%m/%d/%y')
        return (dates+deltas)<datetime.now()
    return True

def callDue(row):
    if 'lastCall' in row and type(row['lastCall']) is str:
        deltas = timedelta(90)
        dates = datetime.strptime(row['lastCall'] or '01/01/00','%m/%d/%y')
        return (dates+deltas)<datetime.now()
    return True

def updateClient(df, row):
    index = row['index']
    colName = row['column']
    value = row['new']
    client = df.iloc[index].to_dict()
    client_id=client['_id']
    client[colName] = value
    # remove added columns
    client.pop('meetingDue',None)
    client.pop('callDue',None)
    
    return collection.replace_one({'_id': client_id}, client)
    
def deleteClient(client):
    global collection
    results = collection.delete_one({'_id': client['_id']})
    return results


In [4]:
results = collection.find()
dfResults = pd.DataFrame(list(results))
dfResults['meetingDue'] = dfResults.apply(lambda row: meetingDue(row), axis=1)
dfResults['callDue'] = dfResults.apply(lambda row: callDue(row), axis=1)

In [5]:
colsToShow = ['lastName','firstName','oppInsurance','oppInvestment','opp401k','oppBenefits']
df = dfResults[colsToShow]

col_defs = { 'lastName': { 'sortable': True, 'editable': False }, 'firstName': { 'sortable': True, 'editable': False }  }
qgWidget = qgrid.show_grid(df, show_toolbar=True)
def onCellEdited(row,widge):
    global dfResults
    result = updateClient(dfResults, row)
    return

def onRowDeleted(row,widge):
    global dfResults
    for index in row['indices']:
        results = deleteClient(dfResults.iloc[index])
        print(results)
    return

qgWidget.on('cell_edited',onCellEdited)
qgWidget.on('row_removed',onRowDeleted)
qgWidget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [6]:
colsToShow = ['lastName','firstName','segment','lastMeeting','lastCall']
dfDates = dfResults[colsToShow]

qgDatesWidget = qgrid.show_grid(dfDates, show_toolbar=True)
def onDfDatesCellEdited(row,widge):
    result = updateClient(dfDates, row)
    return

def onDfDatesRowDeleted(row,widge):
    for index in row['indices']:
        results = deleteClient(dfDates.iloc[index])
        print(results)
    return

def onGenerateReport(b):
    print(b)
    return
wBtn = widgets.Button(description="Generate Report", button_style="info")
wBtn.on_click(onGenerateReport)
def onGenerateReport(b):
    print(b)
    return
wBtn = widgets.Button(description="Generate Report", button_style="info")
wBtn.on_click(onGenerateReport)
display(wBtn)

qgDatesWidget.on('cell_edited',onDfDatesCellEdited)
qgDatesWidget.on('row_removed',onDfDatesRowDeleted)
qgDatesWidget


KeyError: "['lastMeeting', 'lastCall'] not in index"

In [ ]:
## how do we push the new dataframe to the MLAB database

In [ ]:
## how do we generate a report with the required meetings?